In [1]:
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

df = pd.read_csv('lam_data_processed.csv')
df.head()

,Aika,N
0,2020-01-01 00:00:00,12650.0
1,2020-01-01 01:00:00,11666.0
2,2020-01-01 02:00:00,8620.0
3,2020-01-01 03:00:00,6654.0
4,2020-01-01 04:00:00,5784.0


In [6]:
# first slice of data, where the training data is all data before March and test data are first 2 weeks of March
train1 = df[df['Aika'] < '2020-03-01 00:00:00']
train2 = df[df['Aika'] < '2020-03-15 00:00:00']
train3 = df[df['Aika'] < '2020-04-01 00:00:00']
train4 = df[df['Aika'] < '2020-04-15 00:00:00']
train5 = df[df['Aika'] < '2020-05-01 00:00:00']
train6 = df[df['Aika'] < '2020-05-15 00:00:00']
train_data_list = [train1, train2, train3, train4, train5, train6]

In [13]:
# test-data's always the next 2 weeks of real data
test1 = df[(df['Aika'] >= '2020-03-01 00:00:00') & (df['Aika'] < '2020-03-15 00:00:00')]
test2 = df[(df['Aika'] >= '2020-03-15 00:00:00') & (df['Aika'] < '2020-04-01 00:00:00')]
test3 = df[(df['Aika'] >= '2020-04-01 00:00:00') & (df['Aika'] < '2020-04-15 00:00:00')]
test4 = df[(df['Aika'] >= '2020-04-15 00:00:00') & (df['Aika'] < '2020-05-01 00:00:00')]
test5 = df[(df['Aika'] >= '2020-05-01 00:00:00') & (df['Aika'] < '2020-05-15 00:00:00')]
test6 = df[(df['Aika'] >= '2020-05-15 00:00:00') & (df['Aika'] < '2020-06-01 00:00:00')]
test_data_list = [test1, test2, test3, test4, test5, test6]

In [16]:
def pre_modify_data(data_list) :
    for data in data_list :
        data['order'] = data.apply(lambda row: len(data) - row.name, axis=1)
        data['traffic_future'] = data['N'].shift(336) # 14 vuorokautta * 24t => 336t
        data = data.dropna()
    return data_list

pre_modify_data(train_data_list)
pre_modify_data(test_data_list)

/Users/samame/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/samame/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[                     Aika        N  order  traffic_future
 1440  2020-03-01 00:00:00   4182.0  -1104             NaN
 1441  2020-03-01 01:00:00   3008.0  -1105             NaN
 1442  2020-03-01 02:00:00   2416.0  -1106             NaN
 1443  2020-03-01 03:00:00   2021.0  -1107             NaN
 1444  2020-03-01 04:00:00   2091.0  -1108             NaN
 ...                   ...      ...    ...             ...
 1771  2020-03-14 19:00:00  11835.0  -1435             NaN
 1772  2020-03-14 20:00:00   9280.0  -1436             NaN
 1773  2020-03-14 21:00:00   7354.0  -1437             NaN
 1774  2020-03-14 22:00:00   5592.0  -1438             NaN
 1775  2020-03-14 23:00:00   4371.0  -1439             NaN
 
 [336 rows x 4 columns],
                      Aika        N  order  traffic_future
 1776  2020-03-15 00:00:00   3235.0  -1368             NaN
 1777  2020-03-15 01:00:00   2418.0  -1369             NaN
 1778  2020-03-15 02:00:00   1925.0  -1370             NaN
 1779  2020-03-15 03:00:00   

In [17]:
# Muuttujat numpy-muotoon analyyseja varten
X = np.array(train1[['order', 'N']]) # input-muuttujat
y = np.array(train1['traffic_future']) # output-muuttuja
model = linear_model.LinearRegression()
model.fit(X, y)
train1['Ennuste'] = model.predict(X)

/Users/samame/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
